In [21]:
from Preprocessor import Preprocessor

#FilterReview
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import GaussianNB 
from sklearn import metrics 
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import pickle 
import os.path

class SentimentAnalysis:
    def __init__(self):
        
        self.model_filename = 'model/DocVec'
        print('모델을 불러오는중...')
        self.gnb = pickle.load(open('model/finalized_model_gbn.sav', 'rb'))
            
    def input_X(self, para):
        pre = Preprocessor()
        print(para)
        pre.setPara(para)
        print(pre.sentences)
        self.X = pre.transVector()
    def input_Y(self, result):
        self.Y = result
        
    def readCSV(self,filename,tag=True):
        self.data = pd.read_csv(filename,engine='python')
        self.X_content = self.data['sentence'].values
        self.input_X(self.X_content)
        if tag == True:
            self.input_Y(self.data['label'].values)
        
    def trainModel(self,ts):
        self.X_train,self.X_test,self.y_train,self.y_test=train_test_split(self.X,self.Y,test_size=ts)
        self.gnb = GaussianNB()
        self.y_pred = self.gnb.fit(self.X_train, self.y_train).predict(self.X_test)

        # 테스트 socre 확인
        print("Number of right labeled points out of a total %d points : %.2f %%"
      % (len(self.y_test),100*(len(self.y_test)-(self.y_test != self.y_pred).sum())/len(self.y_test)))
        
        pickle.dump(self.gnb, open(self.model_filename+'.sav', 'wb'))
        isSetModel = True
    def predict(self):
        self.y_pred = self.gnb.predict(self.X)
        self.df = pd.DataFrame({'ps_label':self.y_pred, 'sentence':self.X_content})
        return self.y_pred
    def writePredictionResult(self, filename):
        self.df.to_csv(filename,encoding='utf-8-sig',index=False)

In [ ]:
sa = SentimentAnalysis()
sa.readCSV(tag=True,filename='MovieReview/MovieReview.csv')
sa.trainModel(ts=0.25)

모델을 불러오는중...
['아 더빙.. 진짜 짜증나네요 목소리' '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'
 '너무재밓었다그래서보는것을추천한다' ... '그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다'
 '절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네' '마무리는 또 왜이래']
['아 더빙.. 진짜 짜증나네요 목소리' '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'
 '너무재밓었다그래서보는것을추천한다' ... '그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다'
 '절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네' '마무리는 또 왜이래']


In [ ]:
sa = SentimentAnalysis()
sa.readCSV(tag=False,filename='Review.csv')
sa.predict()
sa.writePredictionResult(filename='ReviewScore.csv')

In [ ]:
from collections import namedtuple
from gensim.models import doc2vec
from konlpy.tag import Twitter
import multiprocessing
from pprint import pprint
from gensim.models import Doc2Vec
from sklearn.linear_model import LogisticRegression
import numpy
import pickle


twitter = Twitter()

def read_data(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
    return data

def tokenize(doc):
  # norm, stem은 optional
  return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]


# 테스트 데이터를 읽기
train_data = read_data('data/ratings_train.txt')
test_data = read_data('data/ratings_test.txt')

# 형태소 분류
train_docs = [(tokenize(row[1]), row[2]) for row in train_data[1:100]]
test_docs = [(tokenize(row[1]), row[2]) for row in test_data[1:100]]

# doc2vec 에서 필요한 데이터 형식으로 변경
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_docs]
tagged_test_docs = [TaggedDocument(d, [c]) for d, c in test_docs]


In [3]:
size = 700
# load train data
#doc_vectorizer = Doc2Vec.load('model/doc2vec.model'+str(size))
doc_vectorizer = Doc2Vec.load('modelByUijung/리뷰모음_train_랄라.model')

In [4]:
#infer_vector   :   단어간의 유사도를 나타낸 값
# 분류를 위한 피쳐 생성
train_x = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs]  
train_y = [doc.tags[0] for doc in tagged_train_docs]
test_x = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_test_docs]
test_y = [doc.tags[0] for doc in tagged_test_docs]

In [8]:
doc_vectorizer.infer_vector(tagged_train_docs[0].words)*1000     #

array([ -54.260757  ,   -2.853084  ,  -34.34145   ,   50.7547    ,
        194.22871   ,  -60.40188   ,    2.4805784 ,   11.578267  ,
         84.84678   ,   33.045166  ,  -79.88328   ,  -39.655052  ,
       -134.8376    , -127.83568   ,  -31.927078  ,   -3.9180186 ,
         42.61494   ,   63.66775   ,   69.438576  ,  -29.77148   ,
         68.76532   ,  -12.632942  ,  -77.55052   ,  -60.610287  ,
         83.53197   ,   26.991373  ,  -24.95431   ,  -35.36142   ,
        -27.655022  ,   56.669495  ,  -80.353004  ,   38.236763  ,
         -0.5948937 , -161.06145   ,   36.570663  ,   62.828297  ,
         39.142353  ,   41.932552  ,  -12.516527  ,  -90.4508    ,
        -74.75085   ,   66.50075   ,   98.31159   ,  -36.65413   ,
       -141.14304   ,   45.98348   ,   -5.867824  ,  -19.85603   ,
        -25.722172  ,   53.330933  , -100.416115  ,   47.09637   ,
         81.93322   ,   85.50847   , -120.40288   ,  -36.186207  ,
       -159.84073   ,  -31.598936  ,   46.842815  , -142.52687

In [23]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
pred_y = gnb.fit(train_x, train_y).predict(test_x)

# 테스트 socre 확인
print("Number of mislabeled points out of a total %d points : %d"
      % (len(train_x),(test_y != pred_y).sum()))
# 0.85179
    
'''
classifier = nltk.NaiveBayesClassifier.train(train_xy) #Naive Bayes classifier 적용
print(nltk.classify.accuracy(classifier, test_xy))

'''

Number of mislabeled points out of a total 150000 points : 22232


"\nclassifier = nltk.NaiveBayesClassifier.train(train_xy) #Naive Bayes classifier 적용\nprint(nltk.classify.accuracy(classifier, test_xy))\n\n\n#classifier = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)\nclassifier = LogisticRegression(random_state=1234)\nclassifier.fit(train_x, train_y)\n\n# 테스트 socre 확인\nprint( classifier.score(test_x, test_y) )\n# 0.63904\n\n# save the model to disk\nfilename = 'model/finalized_model.sav'\npickle.dump(classifier, open(filename, 'wb'))\n"

In [19]:
filename = 'model/finalized_model_gbn_'+str(size)+'.sav'
pickle.dump(gnb, open(filename, 'wb'))

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=1234)
classifier.fit(train_x, train_y)

# 테스트 socre 확인
print( classifier.score(test_x, test_y) )
# 0.63904

# save the model to disk
filename = 'model/finalized_model.sav'
pickle.dump(classifier, open(filename, 'wb'))